In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image

In [2]:
query = """
(SELECT 
*, "Regular Season" as Category
FROM
`perceptive-ivy-290216.nba_api_player_pass.pass_made`
)
UNION ALL
(SELECT 
*, "Playoffs" as Category
FROM
`perceptive-ivy-290216.nba_api_player_pass.playoffs_pass_made`)
"""
project_id = "perceptive-ivy-290216"
pass_totals = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
query = """
SELECT PLAYER_ID, TEAM_ABBREVIATION, SEASON FROM `perceptive-ivy-290216.player_team_info.player_bios_regs_totals`
# WHERE season='2023-24'
"""
project_id = "perceptive-ivy-290216"
player_team = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [4]:
def team_color(team):
    if team =="ATL":
        return "#C8102E"
    elif team =="DAL":
        return '#0050B5'
    elif team =="MEM":
        return '#0C2340'
    elif team =="MIA":
        return '#862633'
    elif team =="DEN":
        return '#1d428a'
    elif team =="DET":
        return '#1D4289'
    elif team =="GSW":
        return '#1D4289'
    elif team =="HOU":
        return '#BA0C2F'
    elif team =="IND":
        return '#041E42'
    elif team =="LAC":
        return '#0C2340'
    elif team =="LAL":
        return '#6a00ed'
    elif team =="POR":
        return '#C8102E'
    elif team =="PHX":
        return '#e56020'
    elif team =="PHI":
        return '#1D4289'
    elif team =="ORL":
        return '#0057B7'
    elif team =="OKC":
        return '#0072CE'
    elif team =="NYK":
        return '#FF8200'
    elif team =="NOP":
        return '#85714d'
    elif team =="MIN":
        return '#236192'
    elif team =="MIL":
        return '#2C5234'
    elif team =="BKN":
        return '#707372'
    elif team =="BOS":
        return '#007A33'
    elif team =="SEA":
        return '#00653A'
    elif team =="CHA":
        return '#00778B'
    elif team =="BUF":
        return '#418FDE'
    elif team =="CHI":
        return '#BA0C2F'       
    elif team =="CLE":
        return '#6F263D'
    elif team =="WAS":
        return '#C8102E'
    elif team =="UTA":
        return '#0C2340' 
    elif team =="TOR":
        return '#C8102E'
    elif team =="SAS":
        return '#9EA2A2'
    elif team =="SAC":
        return '#965ddd'                  
    
player_team['color'] = player_team['TEAM_ABBREVIATION'].apply(team_color)

In [5]:
#REGULAR SEASON

In [6]:
pass_totals.columns

Index(['PLAYER_ID', 'PLAYER_NAME_LAST_FIRST', 'TEAM_NAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'PASS_TYPE', 'G', 'PASS_TO',
       'PASS_TEAMMATE_PLAYER_ID', 'FREQUENCY', 'PASS', 'AST', 'FGM', 'FGA',
       'FG_PCT', 'FG2M', 'FG2A', 'FG2_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
       'Season', 'Category'],
      dtype='object')

In [7]:
pass_totals_year=pd.DataFrame(pass_totals.groupby(by=['PLAYER_ID','PLAYER_NAME_LAST_FIRST','Season','Category'])[['G','PASS', 'AST', 'FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A']].sum()).reset_index()
pass_totals_year['FG_PCT']=pass_totals_year['FGM']/pass_totals_year['FGA']
pass_totals_year['FG2_PCT']=pass_totals_year['FG2M']/pass_totals_year['FG2A']
pass_totals_year['FG3_PCT']=pass_totals_year['FG3M']/pass_totals_year['FG3A']

In [8]:
year="2023-24"

In [9]:
#Combine player team data with player aggregated data so that we can color based on team
pass_totals_year_final=pass_totals_year[pass_totals_year["Season"]==year].merge(player_team[player_team["SEASON"]==year], left_on=["PLAYER_ID","Season"], right_on=["PLAYER_ID","SEASON"])
#Filter to regular season
pass_totals_year_final=pass_totals_year_final[(pass_totals_year_final["Category"]=='Regular Season')]
#Create a player and team dict for color for future coloring
player_color_dict_shot_dist=dict(zip(pass_totals_year_final['PLAYER_NAME_LAST_FIRST'], pass_totals_year_final['color']))
team_color_dict_shot_dist=dict(zip(pass_totals_year_final['TEAM_ABBREVIATION'], pass_totals_year_final['color']))

In [10]:
#Remove _ from Column Names
pass_totals_year_final.columns = pass_totals_year_final.columns.str.replace('_', ' ') 
pass_totals_year_final=pass_totals_year_final.fillna(0)
pass_totals_year_final.head()

,PLAYER ID,PLAYER NAME LAST FIRST,Season,Category,G,PASS,AST,FGM,FGA,FG2M,FG2A,FG3M,FG3A,FG PCT,FG2 PCT,FG3 PCT,TEAM ABBREVIATION,SEASON,color
1,2544,"James, LeBron",2023-24,Regular Season,1215,3547,567,661,1346,470,854,191,492,0.491085,0.550351,0.388211,LAL,2023-24,#6a00ed
2,101108,"Paul, Chris",2023-24,Regular Season,1215,2671,384,465,1089,293,598,172,491,0.426997,0.489967,0.350305,GSW,2023-24,#1D4289
4,200768,"Lowry, Kyle",2023-24,Regular Season,2430,2743,239,338,861,236,555,102,306,0.392567,0.425225,0.333333,PHI,2023-24,#1D4289
6,200782,"Tucker, P.J.",2023-24,Regular Season,1539,346,16,36,95,19,46,17,49,0.378947,0.413043,0.346939,LAC,2023-24,#0C2340
8,201142,"Durant, Kevin",2023-24,Regular Season,1440,3255,370,487,1077,344,687,143,390,0.452182,0.500728,0.366667,PHX,2023-24,#e56020


In [18]:
category="AST"

In [19]:
pass_totals_year_final.describe()

,PLAYER ID,G,PASS,AST,FGM,FGA,FG2M,FG2A,FG3M,FG3A,FG PCT,FG2 PCT,FG3 PCT
count,570.0,570.0,570.0,570.0,570.0,570.0,570.0,570.0,570.0,570.0,570.0,570.0,570.0
mean,1377663.145614,1387.166667,1204.075439,113.35614,159.568421,368.987719,106.178947,222.398246,53.389474,146.589474,<NA>,<NA>,<NA>
std,547136.085055,521.211397,1121.039225,132.345136,166.655691,373.281197,110.353416,222.888627,59.269684,156.790503,<NA>,<NA>,<NA>
min,2544.0,81.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<NA>,<NA>,<NA>
25%,1627837.25,1134.0,235.5,16.0,27.0,69.5,18.25,40.25,8.0,26.0,<NA>,<NA>,<NA>
50%,1629724.5,1312.0,885.5,65.5,112.0,265.5,72.5,158.0,34.5,96.5,<NA>,<NA>,<NA>
75%,1630702.75,1558.0,1876.25,159.0,229.5,536.5,155.75,321.25,77.5,209.75,<NA>,<NA>,<NA>
max,1642013.0,3493.0,5843.0,746.0,892.0,1988.0,600.0,1167.0,362.0,919.0,<NA>,<NA>,<NA>


In [20]:
fig_pass = px.bar(
  pass_totals_year_final[pass_totals_year_final["PASS"]>900].sort_values(by=category,ascending=False).head(25),
  x="PLAYER NAME LAST FIRST", 
  y=category,
  color="PLAYER NAME LAST FIRST",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>Player Passing: Made {} For The {} Season</b>".format(category.replace('_', ' ').capitalize(),year),
  hover_data=['PLAYER NAME LAST FIRST', 'Season', 'Category', 'G',
       'PASS', 'AST', 'FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A', 'FG PCT',
       'FG2 PCT', 'FG3 PCT', 'TEAM ABBREVIATION', 'SEASON'], 
  height=800, 
  width=1200,
  )
fig_pass.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pass

In [21]:
save_year=2024

In [22]:
fig_pass.write_html("/Users/rdesh723/statpulse-html/plots/Pass Made/{}/Player {} For The {} Season.html".format(save_year,category.replace('_', ' ').capitalize(),year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOT

In [23]:
category=['FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A', 'FG PCT',
       'FG2 PCT', 'FG3 PCT']
for i in category:
    fig_pass_multi = px.bar(
    pass_totals_year_final[pass_totals_year_final["PASS"]>900].sort_values(by=i,ascending=False).head(25),
    x="PLAYER NAME LAST FIRST", 
    y=i,
    color="PLAYER NAME LAST FIRST",
    color_discrete_map=player_color_dict_shot_dist,     
    template='plotly_white',
    title="<b>Player Passes Made: Receiver {} For The {} Season</b>".format(i.replace('_', ' ').capitalize(),year),
    hover_data=['PLAYER NAME LAST FIRST', 'Season', 'Category', 'G',
        'PASS', 'AST', 'FGM', 'FGA', 'FG2M', 'FG2A', 'FG3M', 'FG3A', 'FG PCT',
        'FG2 PCT', 'FG3 PCT', 'TEAM ABBREVIATION', 'SEASON'], 
    height=800, 
    width=1200,
    )
    fig_pass_multi.update_layout(
        margin=dict(l=5, r=5, t=40, b=5),
        showlegend=False,
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_pass_multi.write_html("/Users/rdesh723/statpulse-html/plots/Pass Made/{}/Player {} For The {} Season.html".format(save_year,i.replace('_', ' ').capitalize(),year),full_html=False, include_plotlyjs='cdn')